<h1><center>Universidade Federal de Minas Gerais</center></h1>
<h3><center>Departamento de Ciência da Computação</center></h3>
<h3><center>Introdução a Banco de Dados -  Trabalho Prático #1</center></h3>


# 1. Introdução

### 1.1. Contexto

Nesse trabalho prático utilizaremos o banco de dados representado pelo schema abaixo:

![Modelo_BD.png](https://github.com/claudiovaliense/ibd/blob/main/data/schema.png?raw=true)

### 1.2. Entrega




1.   Você deve completar esse notebook, executar todas as células e submetê-lo no **Minha UFMG**.
2.   Você pode entregar até o dia 23/01/2022 às 23:59.
3. O trabalho é individual e vale 15 pontos.






# 2. Preparação do Ambiente

In [6]:
# importando os packages necessários
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
import wget

In [2]:
wget.download("https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql")

100% [............................................................................] 277046 / 277046

'despesas_publicas_tp1.sql'

In [3]:
# download do dump do banco de  dados:
!wget https://raw.githubusercontent.com/GiovannaR/ibd/master/data/despesas_publicas_tp1.sql

'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


### 2.1. Criação da conexão com o banco
* No momento da conexão, é informado o nome do arquivo onde será montado o banco de dados. No nosso caso, este arquivo não precisa ser pré-existente, porque iremos carregar as informações de um dump;
* Se este comando for executado com um arquivo que já existe e possui as mesmas tabelas que serão criadas, ocorrerá um erro no momento da execução do script mais à frente.

In [7]:
conn = sqlite3.connect('despesas.db')
cursor = conn.cursor()

### 2.2. Execução do script para inicializar e popular o banco de dados

* Criada a conexão, iremos utilizar o arquivo .sql disponibilizado para recuperar o schema e os dados do banco

In [8]:
f = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
sql = f.read()
cursor.executescript(sql)

### 2.3. Realizando consultas no banco de dados

* Criada a conexão e realizada a carga dos dados no banco, pode ser executada qualquer consulta SQL;
* O SQLite3 permite que o retorno dos dados seja feito em vários formatos, entretanto, utilizaremos o retorno em um dataframe do Pandas.



**CONSULTA EXEMPLO:** todos os órgãos ordenados pela quantidade de orgãos subordinados de forma decrescente.

In [9]:
df = pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

,COD_ORGAO,NOME_ORGAO,QTD_ORGAOS_SUBORDINADOS
0,26000,Ministério da Educação,104
1,30000,Ministério da Justiça e Segurança Pública,6
2,22000,"Ministério da Agricultura, Pecuária e Abastec",5
3,24000,"Ministério da Ciência, Tecnologia, Inovações",4
4,52000,Ministério da Defesa,4
5,53000,Ministério da Integração Nacional,4
6,25000,Ministério da Fazenda,3
7,36000,Ministério da Saúde,3
8,20000,Presidência da República,2
9,39000,"Ministério dos Transportes, Portos e Aviação",2


#3. Consultas

Expresse cada consulta abaixo utilizando SQL: 


**1.** Retorne a quantidade de palavras em cada registro da coluna nome_orgao da tabela orgão.



```
# Isto está formatado como código
```

**2.** Dado a coluna nome_orgao, retorne a primeira posição que a string 'da' aparece da tabela orgão 

**3.** Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

**4.** Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.

> Bloco com recuo



**5.** Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

**6.** Transforme os caracteres em minúsculo e remova todos os termos 'da' na coluna nome_orgao da tabela orgão.

**7.** Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

**8.** Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

**9.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

**10.** Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior valor total em despesas durante o ano de 2018.

**11.** Liste os códigos e nomes dos programas de governo relacionados ao órgão (ou órgãos, caso haja empate) que teve menos registros de despesas públicas durante o ano.

**12.**  Álgebra relacional: projeção do NOME_FUNCAO e NOME_SUBFUNCAO da junção natural das tabelas FUNCAO, DETALHAMENTO_FUNCAO e SUBFUNCAO

**13.**  Álgebra relacional: projeção de MODALIDADE_DESPESA, VALOR_DESPESA e NOME_PROGRAMA_GOVERNO da junção natural das tabelas PROGRAMA_GOVERNO e DESPESA_PUBLICA onde a MODALIDADE_DESPESA = 'Reserva de Contingência'

**14.**  Álgebra relacional: projeção de VALOR_DESPESA e NOME_ORGAO da junção natural das tabelas DESPESA_PUBLICA e ORGAO onde o NOME_ORGAO = 'Ministério da Fazenda'

**15.**  Álgebra relacional: projeção de NOME_PROGRAMA_GOVERNO e VALOR_DESPESA da junção natural das tabelas DESPESA_PUBLICA, PROGRAMA_GOVERNO e ORGAO onde o NOME_ORGAO = 'Ministério da Saúde'

# 4. Fechamento da conexão com o banco

* Após serem realizadas todas as consultas necessárias, é aconselhavel encerrar formalmente a conexão com o banco de dados

In [ ]:
conn.close()